In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - CamP ZipNeRF（Jax）笔记本
<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_camp_zipnerf.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> 在Colab Enterprise中运行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_camp_zipnerf.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> 在GitHub上查看
    </a>
  </td>
</tr></tbody></table>

注意：本笔记本已在以下环境中进行测试：

* Python 版本 = 3.9

## 概述

本笔记本展示了对神经辐射场（Neural Radiance Fields）进行训练和渲染的 [jax 实现](https://github.com/jonbarron/camp_zipnerf)，主要旨在更高效地解决一些传统 NeRF 技术的局限性。传统 NeRF 技术虽然能够通过 2D 图像创建详细的 3D 模型，但计算量大且速度较慢。本实现的主要目的是解决这些问题。

## 目标

在本教程中，您将学习如何：

- 使用[COLMAP](https://colmap.github.io/)执行运动结构（SfM）技术，该技术从一系列二维图像中估计场景的三维结构。
- 使用[Vertex AI自定义作业](https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-custom-job-sample)校准、训练和渲染NERF场景。
- 使用一系列关键帧照片沿着自定义相机路径渲染视频。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI训练
- Vertex AI自定义作业

## 成本

本教程使用Google Cloud的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

安装

In [ ]:
# @title Setup Google Cloud project
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

import os
from datetime import datetime

from google.cloud import aiplatform

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://"  # @param {type:"string"}
assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."

# Create a unique GCS bucket for this notebook, if not specified by the user.
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    shell_output = ! gsutil ls -Lb {BUCKET_URI} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID

staging_bucket = os.path.join(BUCKET_URI, "zipnerf_staging")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=staging_bucket)

# The pre-built calibration docker image.
CALIBRATION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-cloudnerf-calibrate:latest"
# The pre-built training docker image.
TRAINING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-cloudnerf-train:latest"
# The pre-built rendering docker image.
RENDERING_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-cloudnerf-render:latest"

import subprocess
from datetime import datetime
from typing import Any, List

IMAGE_EXTENSIONS = (".png", ".jpg", ".jpeg", ".gif", ".bmp")
GCS_API_ENDPOINT = "https://storage.cloud.google.com/"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def get_mp4_video_link(mp4_rendering_path: str) -> str:
    # Define the gsutil command.
    command = f"gsutil ls {mp4_rendering_path}"

    # Run the command and capture the output.
    try:
        result = subprocess.check_output(command, shell=True, text=True)
        # Split the result by newlines to get a list of files.
        file_list = result.strip().split("\n")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        file_list = []
    mp4_video_link = file_list[0].replace("gs://", GCS_API_ENDPOINT)
    return mp4_video_link


def write_keyframe_list_to_gcs(
    bucket_path: str, output_gcs_file: str, max_files: int = 10
) -> List[Any]:
    # Get the list of files in the GCS bucket.
    cmd = f"gsutil ls {bucket_path}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("Error listing GCS bucket:", result.stderr)
        return []

    # Filter for image files and extract file names.
    files = result.stdout.splitlines()
    image_files = [
        os.path.basename(f) for f in files if f.lower().endswith(IMAGE_EXTENSIONS)
    ]

    output_file = "out.txt"
    with open(output_file, "w") as file:
        for name in image_files[:max_files]:
            file.write(name + "\n")

    cmd = f"gsutil cp {output_file} {output_gcs_file}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print("Error listing GCS bucket:", result.stderr)
        return []

In [ ]:
# @title Prepare dataset
# @markdown Mip-NeRF 360 dataset contains the following 9 scenes:
# @markdown - `bicycle`
# @markdown - `bonsai`
# @markdown - `counter`
# @markdown - `flowers`
# @markdown - `garden`
# @markdown - `kitchen`
# @markdown - `room`
# @markdown - `stump`
# @markdown - `treehill`

# @markdown Please note that `flowers` and `treehill` require author's permission. Each scene comes preprocessed with COLMAP information so the calibration step in the following section is optional.
# @markdown If you need to prepare your dataset and store it on Cloud Storage, then the following example shows how to do this for the [mipnerf360 dataset](https://jonbarron.info/mipnerf360/).


mipnerf_dataset_directory = "mipnerf360_dataset"  # @param {type:"string"}
MIPNERF_DATA_GCS_PATH = os.path.join(BUCKET_URI, mipnerf_dataset_directory)

# Download the bicycle scene data to a local directory.
! rm -rf $mipnerf_dataset_directory
! mkdir -p $mipnerf_dataset_directory
! wget -P $mipnerf_dataset_directory http://storage.googleapis.com/gresearch/refraw360/garden.zip

# Unzip the mipnerf360 garden dataset.
! unzip $mipnerf_dataset_directory/garden.zip -d $mipnerf_dataset_directory

# Move mipnerf360 data from local directory to Cloud Storage.
# This step takes a few minutes to finish.
! gsutil -m cp -R $mipnerf_dataset_directory/* $MIPNERF_DATA_GCS_PATH

## NERF 管道

In [ ]:
# @title Run Camera Pose Estimation Custom Job

# @markdown Once data and experiment paths have been configured, run the custom job below.

# @markdown The following parameters are required:

# @markdown * `use_gpu`: Whether to use GPU or not.
# @markdown * `gcs_dataset_path`: Path to image folder in GCS dataset.
# @markdown * `gcs_experiment_path`: GCS path for storing experiment outputs.
# @markdown * `camera`: Type of camera used. `OPENCV` for perspective, `OPENCV_FISHEYE` for fisheye.

# @markdown The custom job will run on the images in the `gcs_dataset_path` folder and store the colmap outputs in the `gcs_experiment_path/data` folder.

# @markdown On the scenes in this current dataset, this step takes about 30 minutes.

# Folder containing all the images of the garden scene.
# e.g. f"{BUCKET_URI}/{mipnerf_dataset_directory}/garden/images"
INPUT_IMAGES_FOLDER = ""  # @param {type:"string"}

# Folder for storing experiment outputs for calibration, training and rendering.
# e.g. f"{BUCKET_URI}/{mipnerf_dataset_directory}/exp/garden"
OUTPUT_FOLDER = ""  # @param {type:"string"}

# This job will run colmap camera pose estimation.
data_calibration_job_name = get_job_name_with_datetime("colmap")

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": CALIBRATION_DOCKER_URI,
            "args": [
                "-use_gpu",
                "1",
                "-gcs_dataset_path",
                INPUT_IMAGES_FOLDER,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-camera",
                "OPENCV",
            ],
        },
    }
]

data_calibration_custom_job = aiplatform.CustomJob(
    display_name=data_calibration_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

data_calibration_custom_job.run()

In [ ]:
# @title Training the ZipNeRF model

# @markdown Once the Colmap pose calibration is completed, we can run training.

# @markdown The following parameters are required:

# @markdown * `gcs_experiment_path`: GCS path for loading processed dataset and storing experiment outputs.
# @markdown * `factor`: A factor of the downsampled images in the preprocessing step that affects the resolution or detail level of the training pixel ground truth and rendered images. A factor of 2 is recommended for indoor scenes and a factor of 4 for outdoor scenes.

# @markdown The custom job will run on the images in the `gcs_experiment_path/data` colmap dataset and outputs in the checkpoints in `gcs_experiment_path/checkpoints` folder.

# @markdown Depending on the configuration, this step could take up to 3 hours.

# This job will run zipnerf training.

# This is the nerf training job name. You will use it to load the checkpoints
# in the rendering job for the current run.
nerf_training_job_name = get_job_name_with_datetime("nerf_training")

FACTOR = 0  # @param [0, 2, 4, 8]

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": TRAINING_DOCKER_URI,
            "args": [
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-factor",
                str(FACTOR),
            ],
        },
    }
]

nerf_training_custom_job = aiplatform.CustomJob(
    display_name=nerf_training_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_training_custom_job.run(enable_web_access=True)

In [ ]:
# @title Rendering the ZipNeRF model (360)

# @markdown Once the training is completed, we can run rendering.

# @markdown The following parameters are required:

# @markdown * `gcs_experiment_path`: GCS path for loading processed dataset and storing experiment outputs.
# @markdown * `render_video_fps`: Frame rate of rendered video.
# @markdown * `render_path_frames`: Number of frames to render for a path.
# @markdown * `render_resolution`: Standard display resolutions, for example: (VIDEO_WIDTH, VIDEO_HEIGHT).

# @markdown The custom job will run on the images in the `gcs_experiment_path/data` colmap dataset and outputs in the checkpoints in `gcs_experiment_path/checkpoints` folder.

# This job will run zipnerf rendering.
nerf_rendering_job_name = get_job_name_with_datetime("nerf_rendering")
VIDEO_WIDTH = 1280  # @param {type:"integer"}
VIDEO_HEIGHT = 720  # @param {type:"integer"}
RENDER_PATH_FRAMES = 150  # @param {type:"integer"}
RENDER_VIDEO_FPS = 30  # @param {type:"integer"}
VIDEO_RESOLUTION = f"({VIDEO_WIDTH}, {VIDEO_HEIGHT})"

# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": RENDERING_DOCKER_URI,
            "args": [
                "-rendering_job_name",
                nerf_rendering_job_name,
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-render_video_fps",
                str(RENDER_VIDEO_FPS),
                "-render_path_frames",
                str(RENDER_PATH_FRAMES),
                "-render_resolution",
                VIDEO_RESOLUTION,
            ],
        },
    }
]

nerf_rendering_custom_job = aiplatform.CustomJob(
    display_name=nerf_rendering_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_rendering_custom_job.run(enable_web_access=True)

In [ ]:
# @title Show rendered video from GCS

from IPython.display import Video

MP4_RENDERING_PATH = (
    f"{OUTPUT_FOLDER}/render/{nerf_rendering_job_name}/path_videos/videos/*color.mp4"
)
mp4_video_link = get_mp4_video_link(MP4_RENDERING_PATH)
Video(mp4_video_link)

In [ ]:
# @title Rendering the ZipNeRF model (custom camera trajectory)

# @markdown Create keyframe file list for rendering custom camera trajectories.

# @markdown To create a custom camera trajectory in a Neural Radiance Field (NeRF) model using images from the same dataset used for training, you can generate a keyframe file list where each keyframe corresponds to the name of an image file stored in a Google Cloud Storage (GCS) bucket. This section will guide you through creating this keyframe file list.

# @markdown Step 1: Identifying keyframe images
# @markdown First, identify the images within your dataset that you want to use as keyframes. These images should ideally represent the significant views or angles that you want your camera trajectory to include.

# @markdown Step 2: Creating a list of image file names
# @markdown Access Your GCS Bucket: Navigate to your GCS bucket where the dataset is stored.

# @markdown Select Image Files: Choose the specific image files that you want to use as keyframes. Remember, these should be files used in training the NeRF model, as they will have corresponding camera parameters already defined.

# @markdown Compile File Names: Create a list of the file names (not the paths) of these selected images. Ensure that each file name is on a separate line. For example:


# This job will run zipnerf rendering.
nerf_custom_rendering_job_name = get_job_name_with_datetime("nerf_custom_rendering")

# Example usage.
KEYFRAME_IMAGE_FILELIST = (
    f"{OUTPUT_FOLDER}/keyframe_list_{nerf_custom_rendering_job_name}.txt"
)
max_files = 30  # Set this to the number of files you want
write_keyframe_list_to_gcs(
    INPUT_IMAGES_FOLDER, KEYFRAME_IMAGE_FILELIST, max_files=max_files
)

In [ ]:
# @title Run rendering for custom path

# @markdown Once the training is completed, we can run rendering.

# @markdown The following parameters are required:

# @markdown * `gcs_experiment_path`: GCS path for loading processed dataset and storing experiment outputs.
# @markdown * `render_video_fps`: Frame rate of rendered video.
# @markdown * `render_resolution`: Standard display resolutions, for example: (VIDEO_WIDTH, VIDEO_HEIGHT).
# @markdown * `keyframe_image_list`: List of image filename, one per line, for rendering custom camera path.

# @markdown With keyframes, an interpolated path is generated. This path represents a smoothly contoured spline that interconnects the specified keyframe camera poses. The process utilizes a configuration variable, `render_spline_n_interp`, which is preset to a default value of 30. As a result, the finalized interpolated path comprises a total of `render_spline_n_interp` * (n - 1) poses. In the specific scenario under discussion, the config.render_spline_n_interp is configured to 30. **With an input of 30 keyframes, the calculation yields a total of 30 * 29, amounting to 870 poses**.

# This job will run zipnerf rendering.
# Worker pool spec.
machine_type = "n1-highmem-64"
num_nodes = 1
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 8
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": gpu_type,
            "accelerator_count": num_gpus,
        },
        "replica_count": num_nodes,
        "container_spec": {
            "image_uri": RENDERING_DOCKER_URI,
            "args": [
                "-rendering_job_name",
                nerf_custom_rendering_job_name,
                "-training_job_name",
                nerf_training_job_name,
                "-gcs_experiment_path",
                OUTPUT_FOLDER,
                "-render_resolution",
                VIDEO_RESOLUTION,
                "-gcs_keyframes_file",
                KEYFRAME_IMAGE_FILELIST,
            ],
        },
    }
]

nerf_custom_rendering_custom_job = aiplatform.CustomJob(
    display_name=nerf_custom_rendering_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=staging_bucket,
)

nerf_custom_rendering_custom_job.run(enable_web_access=True)

In [ ]:
# @title Show rendered video from GCS

from IPython.display import Video

MP4_RENDERING_PATH = (
    f"{OUTPUT_FOLDER}/render/{nerf_rendering_job_name}/path_videos/videos/*color.mp4"
)
mp4_video_link = get_mp4_video_link(MP4_RENDERING_PATH)
Video(mp4_video_link)

In [ ]:
# @title Clean up resources
# @markdown Delete the experiment finished jobs and bucket to avoid
# @markdown unnecessary continouous charges that may incur.

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

# Delete pose estimation, training and rendering custom jobs.
if data_calibration_custom_job.list(
    filter=f'display_name="{data_calibration_job_name}"'
):
    data_calibration_custom_job.delete()
if nerf_training_custom_job.list(filter=f'display_name="{nerf_training_job_name}"'):
    nerf_training_custom_job.delete()
if nerf_rendering_custom_job.list(filter=f'display_name="{nerf_rendering_job_name}"'):
    nerf_rendering_custom_job.delete()
if nerf_custom_rendering_custom_job.list(
    filter=f'display_name="{nerf_custom_rendering_job_name}"'
):
    nerf_custom_rendering_custom_job.delete()